## Chapter 7

7장에서는 함수 데커레이터와 클로저에 관해 배운다. 함수 데커레이터는 소스코드의 함수를 표시해서 함수 작동을 개선할 수 있게 해주는 좋은 기능이다. 이는 지역변수와 관련된 개념인 클로저와도 관련이 깊다. 먼저 데커레이터에 관해 알아본다.

+ 데커레이터

데커레이터는 다른 함수를 인수로 받는 콜러블이다. 기능은 데커레이트된 함수에 어떤 처리를 수행하고 나서 함수를 반환하거나, 함수를 다른 함수나 콜러블 객체로 대신하는 것이다. 다음 두 코드는 함수를 다른 함수로 대체하는, 똑같은 기능을 한다.

In [4]:
def decorate(func):
  def inner():
    print("decorator running")
  return inner

@decorate
def target():
  print('running target')

target()

decorator running


In [6]:
def target():
  print('running target')

target=decorate(target)
target()

decorator running


즉 데커레이터를 이용하면 함수를 다른 함수로 대체할 수 있다는 것이다. 이는 데커레이터 함수가 함수 객체를 반환하게 할 때 아주 명확해진다. 

위의 코드에서는 target=decorate(target) 이 되어 target=inner 가 되고 따라서 target함수는 inner 함수로 대체된다. 물론 이는 함수 객체를 인수로 전달해 호출하는 것과 다를 바가 없다. 즉 문법적 설탕(syntactic sugar. 프로그래밍 언어 차원에서 제공하는 더 간결한 논리적 표현. C의 삼항연산자 등이 있다)이다. 다른 표현으로 대체 가능한 것이다. 그러나 런타임에 프로그램을 변경하는 코드(메타 프로그래밍)를 짤 때 데커레이터는 상당히 편리하며 더 간결한 코드를 짜는 것을 돕는다.

1. 데커레이터는 모듈이 로딩될 때 바로 실행된다
2. 데커레이터는 데커레이트된 함수를 다른 함수로 대체하는 기능이 있다

+ 데커레이터의 실행 시점

데커레이터 핵심 특징은 데커레이트된 함수가 정의된 직후에 구문이 실행된다는 것이다. 이는 일반적으로, 파이썬이 모듈을 로딩하는 시점인 임포트 타임에 실행된다. 이는 코드가 한 줄 한 줄 실행되는 '런타임' 과는 다른 개념이다. 

즉 데커레이터는 모듈 내에 어떤 함수보다도 먼저 실행되는 것이다(임포트 타임). 단 임포트 타임에는 데커레이터 구문(즉 func=decorate(func))이 실행되는 것이고, '데커레이트된 함수'가 실행되는 것과는 다르다.


+ 데커레이터 적용시의 규칙

데커레이터가 실제 코드에서 사용될 때는 보통 다음 두 가지가 지켜진다.
1. 데커레이터를 정의하는 모듈과 데커레이터를 적용할 함수가 존재하는 모듈이 따로 있다.
2. 데커레이터가 반환하는 함수는 보통 데커레이터 함수 내부에서 정의된 함수이다. 파이썬은 함수 내부에 함수를 정의하는 것을 허용한다.

+ 데커레이터로 코드 개선하기

이 데커레이터를 이용하면 6장에서 보았던 할인 함수 코드들을 더 간단하게 짤 수 있다. 앞의 promotion 함수들을 생각해 보자. 

In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer=namedtuple('Customer', 'name fidelity')


class LineItem: #구입한 물건
    def __init__(self, product, quantity, price):
        self.product=product
        self.quantity=quantity
        self.price=price

    def total(self):
        return self.price*self.quantity


class Order:
    def __init__(self, customer, cart, promotion=None): #Customer 튜플 받음
        self.customer=customer
        self.cart=list(cart) #LineItem들이 들어 있음
        self.promotion=promotion #적용되는 할인. 기본 할인은 없다

    def total(self):
        if not hasattr(self, '__total'):
            self.__total=sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount=0
        else:
            discount=self.promotion(self) #깎이는 비용
        return self.total()-discount

    def __repr__(self):
        fmt='<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    return order.total()*0.05 if order.customer.fidelity>=1000 else 0


def bulkitem_promo(order):
    dc=0
    for item in order.cart:
      if item.quantity>=20:
          dc+=item.total()*0.1
    return dc


def largeorder_promo(self, order):
    distinct_items={item.product for item in order.cart} #set을 이용해 상품 종류 계산
    if len(distinct_items)>=10:
        return order.total*0.07
    return 0


promos=[fidelity_promo, bulkitem_promo, largeorder_promo]


def best_promo(order):
    return max(promo(order) for promo in promos)

위의 코드는 다양한 할인 조건들을 정의하고, 손님이 산 물건들을 탐색해서 적용할 수 있는 가장 좋은 할인 조건을 적용해 주는 best_promo 함수를 정의하는 코드다. 그런데 만약에 새로운 할인 함수를 추가했는데 이를 promos 리스트에 그 함수를 추가하지 않았다면 best_promo함수는 새롭게 추가된 할인 함수를 무시해 버린다. 데커레이터를 활용하면 이런 문제를 어느 정도 해결할 수 있다. 다음과 같이 할인 함수들을 바꾸는 것이다.

In [ ]:
promos=[]


def promotion(promo_func): #promos 리스트에 함수를 추가해 준 후 그 함수를 그대로 리턴한다
    promos.append(promo_func)
    return promo_func


@promotion
def fidelity_promo(order):
    return order.total()*0.05 if order.customer.fidelity>=1000 else 0


@promotion
def bulkitem_promo(order):
    dc=0
    for item in order.cart:
      if item.quantity>=20:
          dc+=item.total()*0.1
    return dc


@promotion
def largeorder_promo(self, order):
    distinct_items={item.product for item in order.cart} #set을 이용해 상품 종류 계산
    if len(distinct_items)>=10:
        return order.total*0.07
    return 0


def best_promo(order):
    return max(promo(order) for promo in promos)

위 코드를 보면 promotion 데커레이트 함수는 할인 함수를 인자로 받아서 promos 리스트에 넣고, 인자로 받았던 할인 함수를 리턴한다. 이렇게 함수를 받아서 그 함수 객체로 특정 행위(리스트에 추가하는 등)를 수행한 후 인자로 받은 함수를 그대로 리턴해 주는 데커레이터를 '등록 데커레이터' 라고 한다. 아무튼 위 코드와 같이 코드를 짜면 간단하게 할인 함수들의 리스트를 만들고, 그 중에 가장 할인율이 높은 함수를 적용할 수 있다. 이런 데커레이터를 사용하는 방식은 여러가지 이점이 있다.

1. 전략 함수(여기서는 할인함수)의 이름이 _promo로 끝날 필요가 없다. 이는 globals() 함수를 사용할 경우 필요한 부분이었다.
2. 임시로 할인 함수를 배제하기가 정말 쉽다. 그냥 등록 데커레이터 부분만 주석 처리하면 그 함수를 promos 리스트에 등록되지 않도록 할 수 있다.
3. 할인 전략을 구현한 함수는 @promotion 데커레이터가 적용되는 한 어느 모듈에서든 정의할 수 있다.

+ 변수의 범위

데커레이터를 이용하면 데커레이트된 함수를 변경할 수 있다는 것을 배웠다. 단순히 데커레이터 내부에 내부 함수를 정의하고, 그 함수 객체를 반환하도록 하면 된다. 이런 경우 코드는 거의 언제나 '클로저closure'개념에 의존하는데 먼저 그것부터 알아보자. 이는 변수 범위의 작동 방식과 큰 연관이 있다. 다음과 같은 코드를 보자.

In [7]:
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: ignored

이는 당연히 에러다. 변수 b가 정의되지 않았는데 함수 f1의 내부에서는 b를 호출하고 있기 때문이다. 만약 함수 외부 혹은 내부에서 먼저 b를 할당해 줬다면 함수는 정상적으로 실행될 것이다. 당연하다. 그런데 다음과 같은 코드를 보자.

In [9]:
b=10
def f2(a):
  print(a)
  print(b)
  b=30

f2(50)

50


UnboundLocalError: ignored

이는 언뜻 보기에 왜 에러인지 알 수 없다. 이미 전역 변수 b를 할당해 주지 않았나? f2함수 내부에서는 전역에서 선언된 b를 호출해서 출력하면 되는데 어째서 지역 변수 b가 아직 선언되지 않았다는 에러가 뜨는 것일까? 이는 파이썬에서 함수를 컴파일할 때 함수 안에서 할당된 변수는 지역 변수로 판단하기 때문이다. 

그런데 파이썬은 인터프리터 언어인데 어째서 함수를 '컴파일' 하는가? 이는 파이썬이 온전히 인터프리터 언어로 동작하지 않기 때문이다. 파이썬은 실행될 떄 먼저 Cpython 이라는 구현체를 거치게 되는데 이건 파이썬 코드를 먼저 바이트코드로 변환한 후 인터프리터를 통해 실행한다.(https://cjh5414.github.io/about-python-and-how-python-works/)

아무튼 실제로 위 코드의 f2는 지역 환경(함수 내부)에서 변수 b를 가져오려는 동작을 수행한다. 하지만 b를 프린트하는 시점에서 b에는 아직 값이 할당되어 있지 않기에 에러가 발생하는 것이다. 이떄 전역 변수 b는 고려조차 하지 않는다. 만약 함수 내부에 b를 할당하는 문장이 있지만 b를 전역 변수로 쓰고 싶다면 global 키워드를 이용해 global b 를 선언해 줘야 한다.

+ 클로저

이제 클로저 개념에 대해 알아보자. 이는 예제를 통해 알아보는 것이 좋다. 먼저 다음과 같은 함수를 보자.


In [ ]:
def make_averager():
  series=[]

  def averager(new_value):
    series.append(new_value)
    total=sum(series)
    return total/len(series)
  
  return averager

avg=make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


함수를 이해하는 방식은 간단하다. make_averager 함수는 호출되면 average 함수를 호출하고 average 함수는 series리스트에 받은 인수를 추가하고 현재까지의 평균을 계산해 출력한다. 그런데 이 avg() 함수는 어디서 series 리스트를 호출하는 것일까? avg(val) 을 호출할 때 make_average 함수는 이미 리턴을 끝냈으므로 지역 범위도 이미 사라졌다. 하지만 언뜻 보기에 series 리스트는 함수의 지역 범위 내에 있다. 그럼 대체 series 리스트는 어디서 오는 것인가?

+ 자유 변수

답은 series가 자유 변수free variable, 즉 지역 범위에 바인딩되어 있지 않은 변수라는 것이다. 이때 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수가 바로 클로저이다. 따라서 함수를 정의하던 지역 범위가 사라진 후에도 함수 호출을 통해 자유 변수에 접근할 수 있다.

*함수가 비전역 외부 변수를 다루는 경우는 그 함수가 다른 함수 안에 정의된 경우뿐임에 주의하라. 즉 자유 변수는 어쨌든 하나의 함수 내에 존재해야 한다.

그런데 만약 함수 내에서 변수에 새로운 객체를 할당하게 되면, 즉 그 변수가 참조하고 있는 객체를 다른 것으로 변경하게 되면, 그 변수는 자유 변수가 아니라 함수 내부의 지역 변수로 취급되게 된다. 위의 코드 같은 경우에는 `append` 가 참조 객체의 속성을 변경할 뿐 참조하고 있는 객체를 다른 것으로 바꾸지는 않으므로 잘 작동했던 것이다.

다음과 같은 코드를 보자. 

In [ ]:
def make_averager():
  count=0
  total=0

  def average(new_value):
    count+=1
    total+=new_value
    return total/count
  return average

avg=make_averager()
print(avg(10))
print(avg(11))

UnboundLocalError: ignored

에러가 발생한다. count+=1은 실제로 count=count+1 이기 때문에, average 함수 내부에서 변수 할당이 새로 일어나고 count변수는 average 함수 내부의 지역 변수가 되는 것이다. 이를 방지하기 위한 nonlocal 키워드가 파이썬3에 등장한다. nonlocal 키워드는 변수가 자유 변수임을 나타내 주며 이 선언이 되었을 경우 변수에 새로운 값이 할당되더라도 그 변수는 자유 변수이다. 가령 위의 코드를 nonlocal을 이용해서 아래와 같이 고치면 잘 작동한다.

In [10]:
def make_averager():
  count=0
  total=0

  def average(new_value):
    nonlocal count, total
    count+=1
    total+=new_value
    return total/count
  return average

avg=make_averager()
print(avg(10))
print(avg(11))

10.0
10.5


또 파이썬에서는 메서드 데커레이터를 잘 쓰기 위한 여러 가지 내장 데커레이터와 함수를 지원한다. 대표적으로는 함수의 signature 보존을 돕는 functools.wraps() 와(https://brownbears.tistory.com/239) functools.lru_cache(), functools.singledispatch() 등이 있다. 여기서는 lru_cache 와 singledispatch 에 대해 알아본다. 일단 간단한 시간 측정 데커레이터를 만들자.

In [ ]:
import time

def clock(func):
  def clocked(*args):
    t0=time.perf_counter()
    result=func(*args)
    elapsed=time.perf_counter()
    name=func.__name__
    arg_str=", ".join(repr(arg) for arg in args)
    print('[%0.8f초] %s(%s) -> %r'%(elapsed-t0, name, arg_str, result))
    return result
  return clocked


@clock
def snooze(sec):
  time.sleep(sec)

@clock
def factorial(n):
  return 1 if n<2 else n*factorial(n-1)

snooze(0.123)
print(factorial(5))


[0.12323454초] snooze(0.123) -> None
[0.00000053초] factorial(1) -> 1
[0.00003997초] factorial(2) -> 2
[0.00006061초] factorial(3) -> 6
[0.00075555초] factorial(4) -> 24
[0.00079859초] factorial(5) -> 120
120


+ lru_cache 의 사용

위 코드는 매우 전형적인 데커레이터의 작동 방식을 보여준다. 데커레이트된 함수를 동일한 인수를 받는 함수로 교체하고, 데커레이트된 함수가 반환해야 할 값을 반환하면서 추가적인 처리를 수행한다. 이제 이 시간 측정 데커레이터를 사용해 lru_cache 를 사용해 보자. 이는 이전에 실행한 함수의 결과를 저장함으로써 메모이제이션을 수행한다. 이 메모이제이션은 엄청난 속도 향상을 가져온다는 사실은 dp 알고리즘 문제를 풀어본 사람이라면 누구나 알 수 있다.

이때 이름에 LRU가 붙은 이유는 Least Recently Used의 약자로서 오랫동안 사용되지 않은 항목을 버림으로써 캐시의 무한정 확장을 막는다는 뜻이다.

다음과 같은 느린 피보나치 수열 생성 코드를 보자.

In [ ]:
@clock
def fibo(n):
  if n<2:
    return n
  return fibo(n-1)+fibo(n-2)

print(fibo(10))

[0.00000062초] fibo(1) -> 1
[0.00000057초] fibo(0) -> 0
[0.00011372초] fibo(2) -> 1
[0.00000052초] fibo(1) -> 1
[0.00016966초] fibo(3) -> 2
[0.00000045초] fibo(1) -> 1
[0.00000045초] fibo(0) -> 0
[0.00005212초] fibo(2) -> 1
[0.00027163초] fibo(4) -> 3
[0.00000034초] fibo(1) -> 1
[0.00000043초] fibo(0) -> 0
[0.00005102초] fibo(2) -> 1
[0.00000041초] fibo(1) -> 1
[0.00011533초] fibo(3) -> 2
[0.00043540초] fibo(5) -> 5
[0.00000034초] fibo(1) -> 1
[0.00000038초] fibo(0) -> 0
[0.00007577초] fibo(2) -> 1
[0.00000045초] fibo(1) -> 1
[0.00016796초] fibo(3) -> 2
[0.00000035초] fibo(1) -> 1
[0.00000042초] fibo(0) -> 0
[0.00003355초] fibo(2) -> 1
[0.00025794초] fibo(4) -> 3
[0.00072995초] fibo(6) -> 8
[0.00000032초] fibo(1) -> 1
[0.00000043초] fibo(0) -> 0
[0.00009383초] fibo(2) -> 1
[0.00000049초] fibo(1) -> 1
[0.00018849초] fibo(3) -> 2
[0.00000032초] fibo(1) -> 1
[0.00000045초] fibo(0) -> 0
[0.00005191초] fibo(2) -> 1
[0.00029364초] fibo(4) -> 3
[0.00000035초] fibo(1) -> 1
[0.00000043초] fibo(0) -> 0
[0.00005341초] fibo(2) -> 1
[

In [ ]:
import functools

@functools.lru_cache()
@clock
def fibo(n):
  if n<2:return n
  return fibo(n-1)+fibo(n-2)

print(fibo(10))

[0.00000058초] fibo(1) -> 1
[0.00000052초] fibo(0) -> 0
[0.00009821초] fibo(2) -> 1
[0.00011696초] fibo(3) -> 2
[0.00013436초] fibo(4) -> 3
[0.00015258초] fibo(5) -> 5
[0.00016979초] fibo(6) -> 8
[0.00018601초] fibo(7) -> 13
[0.00020316초] fibo(8) -> 21
[0.00022089초] fibo(9) -> 34
[0.00024464초] fibo(10) -> 55
55


위의 두 코드를 보면 캐시를 이용해 메모이제이션을 진행한 코드가 훨씬 빠르게 동작한다는 것을 알 수 있다. 게다가 fibo() 함수의 재귀 호출 횟수도 훨씬 줄었다. 이때 주의할 점이 두 가지 있다.

1. lru_cache(maxsize=128, typed=False) 가 함수의 기본형이다. maxsize는 얼마나 많은 호출을 저장할지(즉 메모이제이션에 얼마나 많은 메모리를 사용할지)를 결정한다. 캐시가 가득차면 가장 오래된 결과를 버리고 공간을 확보하는 것이다. 최적의 성능을 위해서는 maxsize가 2의 제곱꼴이 되어야 한다(아마 amortized O(1)로 메모리 관리를 하는 듯) typed인수가 True일 경우 인수 자료형이 다르면 결과를 다르게 저장한다(1과 1.0을 다르게 취급하는 등)

2. lru_cache는 결과 저장을 위해 딕셔너리를 사용하므로 데커레이트된 함수가 받는 인수는 모두 해시 가능해야 한다.


+ functools.singledispatch

이제 파이썬에서 제네릭 함수를 사용할 수 있게 해주는 functools.singledispatch 데커레이터에 대해 알아보자.
원래 파이썬에서는 메서드 오버로딩을 지원하지 않으므로 서로 다르게 처리하고자 하는 자료형별로 새로운 함수를 선언하였다. 그러나 이는 여러 가지 불편한 부분이 많았고 파이썬3.4부터는 singledispatch 데커레이터가 등장해, @singledispatch로 일반 함수를 데커레이트할 시 범용 함수로 사용할 수 있게 되었다. 즉 함수가 첫 번째 인수의 자료형에 따라 다른 방식의 연산을 수행하게 되는 것이다.
*그러나 singledispatch는 메서드 오버로딩 지원을 위해 설계된 것이 아니다. singledispatch는 단일 클래스나 디스패치 함수에 너무 많은 책임을 부여하지 않도록 하는 모듈화된 확장을 위한 설계이다.

가령 다음과 같은 코드를 보자

In [11]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
  content=html.escape(repr(obj))
  return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
  content=html.escape(text).replace('\n', '<br>\n')
  return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
  return '<pre>{0} (0x{0:x})</pre>'.format(n)

print(htmlize('witch'))
print(htmlize(10003482))

<p>witch</p>
<pre>10003482 (0x98a41a)</pre>


위와 같이 htmlize에 어떤 자료형의 인수가 들어가는지에 따라서 출력값이 달라지는 것이다. 물론 각각의 특화된 함수는 이름이 의미가 없으므로 _로 이름을 정해준다.

그리고 가능하면 singledispatch를 사용할 때 int나 list 같은 구상 클래스보다는 numbers.integral이나 abc.MutableSequence같은 추상 베이스 클래스를 처리하도록 특화된 함수를 등록하는 게 좋다. 그러면 호환되는 자료형을 더 폭넓게 지원할 수 있기 때문이다.

이런 식으로 singledispatch를 이용해 제네릭 함수를 사용하게 되면 코드의 유지보수, 확장 그리고 협업에 많은 장점이 있다. 

+ 매개변수화된 데커레이터

또한 데커레이터 팩토리를 이용하면 데커레이트 함수에도 인수를 적용할 수 있다. 가령 함수를 활성화/비활성화하는 걸 결정하는 active인수를 받는 register 함수 코드를 보자.

In [ ]:
registry=set()

def register(active=True): #인수 전달을 위한 데커레이터 팩토리
  def decorate(func):
    print('running register(active=%s) -> decorate(%s)' % (active,func))
    if active:
      registry.add(func)
    else:
      registry.discard(func)
    return func
  return decorate

@register(False)
def f1():
  print('run f1()')

@register()
def f2():
  print('run f2()')

f1()
f2()
print(registry)

running register(active=False) -> decorate(<function f1 at 0x7f705cec88c8>)
running register(active=True) -> decorate(<function f2 at 0x7f705cfe79d8>)
run f1()
run f2()
{<function f2 at 0x7f705cfe79d8>}


위 코드의 실행 결과를 보면 register 함수에 False인수를 전달한 f1()은 registry에 추가되지 않은 걸 확인할 수 있다. 이와 같이 데커레이터 팩토리를 이용해 데커레이터 함수에도 인수를 전달 가능하다.